In [11]:
import pandas as pd
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")

In [12]:
import sys
sys.path.append("../utils/")

In [13]:
import pipeline_helpers as ph

### Pipeline to evaluate the default hyperparameters on the test set
- Update pool price 12 steps at a time.

In [14]:
X_train = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/train/X_train.csv",
    parse_dates=["date"],
    index_col="date",
)

y_train = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/train/y_train.csv",
    parse_dates=["date"],
    index_col="date",
)

X_train = X_train.sort_values(by="date")
X_train = X_train.asfreq("H")
y_train = y_train.sort_values(by="date")
y_train = y_train.asfreq("H")

In [15]:
X_test = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/test/X_test.csv",
    parse_dates=["date"],
    index_col="date",
)

y_test = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/test/y_test.csv",
    parse_dates=["date"],
    index_col="date",
)

X_test = X_test.sort_values(by="date")
X_test = X_test.asfreq("H")
y_test = y_test.sort_values(by="date")
y_test = y_test.asfreq("H")

In [16]:
X_test = X_test[:"2023-02-03"]
y_test = y_test[:"2023-02-03"]

In [17]:
lgbm_pipeline = ph.initialize_default_lgbm_forecaster(n_estimators=1)
fh = ForecastingHorizon(np.arange(1, 12 + 1))

In [18]:
forecast_len = 12
step_length = 12

In [19]:
lgbm_pipeline.fit(y=y_train, X=X_train, fh=fh)

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('forecast',
                                                                DirectTabularRegressionForecaster(estimator=LGBMRegressor(device='gpu', n_estimators=1),
                                                                                                  window_length=24))]))])

In [20]:
rolling_prediction_df = ph.get_rolling_predictions(lgbm_pipeline, X_train, X_test, y_test, fh, step_length, forecast_len, verbose=False)

In [21]:
fold_actuals, fold_predictions_list, rmse_list = ph.get_fold_predictions(rolling_prediction_df, y_test)

In [24]:
y_hist = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/filtered_target_medium.csv",
    parse_dates=["date"],
    index_col="date",
)

y_hist = y_hist.sort_values(by="date")
y_hist = y_hist.asfreq("H")

In [25]:
ddf = ph.get_plotting_df(fold_actuals=fold_actuals, fold_predictions_list=fold_predictions_list, y_hist=y_hist)

In [26]:
fig = px.line(ddf, x="periodstep", y=["HistoricalPrice", "FuturePrice", "Predicted"], animation_frame="timestep")
fig.update_layout(height=700)  
fig.show()

In [27]:
ph.get_aeso_predictions(y_test.index[0], y_test.index[-1])

RMSE for the predictions by AESO for the same time period as the test set: 51.68 CAD/MWh
